In [ ]:
# default_exp dlc_importer

# DLCImporter

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *
from fastcore.imports import operator

In [ ]:
#export
import pandas as pd

class DLCImporter:
    """Used to import DLC result files."""
    
    def import_hdf(self, file):
        df = pd.read_hdf(file)
        df.columns = df.columns.droplevel(0) # drop redundant scorer
        return df

`DLCImporter` can be used to import result files from DeepLabCut in H5 format.
It will automatically drop the unnecessary `scorer` level.

In [ ]:
imp = DLCImporter()
df = imp.import_hdf('example_data/coordinates.h5')
display(df)

bodyparts        head                               beak              \
coords              x           y likelihood           x           y   
0          773.376465  231.518768   0.999999  726.495178  235.638046   
1          773.129822  231.487213   0.999999  725.662231  235.242844   
2          773.009827  231.793518   0.999999  726.025696  235.272522   
3          773.748779  231.791260   0.999999  726.288940  235.864319   
4          774.934326  231.623734   0.999999  726.298279  235.749908   
..                ...         ...        ...         ...         ...   
95         691.788513  232.490265   1.000000  673.796082  238.801743   
96         691.545410  232.707413   1.000000  673.634888  238.658234   
97         691.117371  232.242767   1.000000  673.748840  239.055954   
98         691.294067  232.225220   1.000000  673.927002  239.141891   
99         691.483643  232.269226   1.000000  673.797241  239.390625   

bodyparts              left_neck                         right_neck  ...  \
coords    likelihood           x           y likelihood           x  ...   
0           0.999981  726.502014  277.634125   0.999998  803.271179  ...   
1           0.999951  725.964478  278.003082   0.999999  803.197144  ...   
2           0.999978  725.764893  278.884918   0.999998  802.567810  ...   
3           0.999985  725.889465  279.045715   0.999998  803.356934  ...   
4           0.999990  726.302551  278.388367   0.999999  802.530273  ...   
..               ...         ...         ...        ...         ...  ...   
95          0.018886  697.399841  282.134796   0.999998  737.725342  ...   
96          0.016135  697.256165  283.058899   0.999999  736.505920  ...   
97          0.007289  696.269043  282.351929   0.999999  735.976685  ...   
98          0.004682  695.629456  282.407013   1.000000  735.639404  ...   
99          0.010126  695.367371  281.720947   0.999999  735.199585  ...   

bodyparts         b1          b2                                 b3  \
coords    likelihood           x           y likelihood           x   
0           0.999986  731.942932  212.751831   0.999972  637.101440   
1           0.999984  732.492004  212.608414   0.999969  636.351440   
2           0.999989  732.367859  212.449570   0.999967  636.567993   
3           0.999988  732.378052  212.362946   0.999962  636.978821   
4           0.999991  732.364014  212.297073   0.999969  637.059021   
..               ...         ...         ...        ...         ...   
95          0.999981  729.320374  214.642715   0.999988  636.506165   
96          0.999983  729.624817  215.617279   0.999986  636.087585   
97          0.999982  729.727722  214.848831   0.999987  636.203613   
98          0.999983  729.409668  214.924347   0.999989  636.108765   
99          0.999981  729.390991  215.064926   0.999983  636.145447   

bodyparts                                 b4                         
coords              y likelihood           x           y likelihood  
0          275.546967   0.999044  720.761475  260.645416   0.004323  
1          274.908783   0.999068  720.861084  260.821045   0.003748  
2          275.915558   0.999465  721.077087  261.231415   0.003017  
3          276.222534   0.999626  721.093933  261.198242   0.004915  
4          276.169647   0.999717  720.964783  260.974213   0.003497  
..                ...        ...         ...         ...        ...  
95         275.310120   0.999486  742.268372  243.916214   0.001441  
96         275.022858   0.999226  741.823853  243.833771   0.001208  
97         275.289307   0.999269  741.812866  243.337326   0.000828  
98         274.952759   0.999238  987.140259  601.538696   0.000869  
99         274.386780   0.999225  742.091919  245.363281   0.000920  

[100 rows x 60 columns]

For further processing and analyzes, it can be useful to transform the tracked coordinates into relative coordinates (being relative to a specified bodypart). This can be done by specifying a bodypart as the relative origin.


In [ ]:
#export
def transform_to_relative(df, bodypart):
    df_rel = df.copy()

    for bp in df.columns.levels[0]:
        df_rel[bp, 'x'] = df[bp, 'x'] - df[bodypart, 'x']
        df_rel[bp, 'y'] = df[bp, 'y'] - df[bodypart, 'y']

    return df_rel

In [ ]:
df_relative = transform_to_relative(df, 'body')

test_close(df_relative['head','x'][0], -30.6, 0.1)
test_close(df_relative['head','y'][0], -119.1, 0.1)
test_close(df_relative['tail','x'][0], 70.87, 0.1)
test_close(df_relative['tail','y'][0], 135.08, 0.1)

By using the `arctan2` function of Numpy, we are able to calculate the angle of the vector from the origin to a specific point.

In [ ]:
import numpy as np

y = np.array([1, 1, 1, -1, 0.75])
x = np.array([0, 1, -1, 1, 1.5])

result = np.degrees(np.arctan2(y, x))
expected = np.array([90, 45, 135, -45, 26.565051])

np.testing.assert_almost_equal(result, expected, decimal=3)

In [ ]:
# export
def add_middle_neck(df):
    df_middle_neck = df.copy()
    df_middle_neck['middle_neck', 'x'] = (df_middle_neck['left_neck','x'] + df_middle_neck['right_neck', 'x']) / 2
    df_middle_neck['middle_neck', 'y'] = (df_middle_neck['left_neck','y'] + df_middle_neck['right_neck', 'y']) / 2
    return df_middle_neck

Since we already moved the body into the origin, we can now decide for a good point for determining the angle of the pigeon rotation.
The vector between the body and middle point between left_neck and right_neck seems to be suitable.
Therefore, we first calculate this point, we can later use to determine the rotation of the pigeon.

In [ ]:
# assume we have pandas Dataframe
df_example = df.copy()
df_example['left_neck', 'x'][0] = 1
df_example['left_neck', 'y'][0] = 1
df_example['right_neck', 'x'][0] = 2
df_example['right_neck', 'y'][0] = 0.5

df_result = add_middle_neck(df_example)

test_close(df_result['middle_neck','x'][0], 1.5, 0.1)
test_close(df_result['middle_neck','y'][0], 0.75, 0.1)


Based on the `middle_neck` coordinates, we now add the corresponding rotation to the dataframe.

In [ ]:
# export
def add_rotation(df):
    df = df.copy()
    df['rotation_angle'] = np.degrees(np.arctan2(df['middle_neck', 'y'], df['middle_neck', 'x']))
    return df

In [ ]:
df_result = add_rotation(df_result)
test_close(df_result['rotation_angle'][0], 26.565051, 0.1)

In [ ]:
# hide
def _rotate_row(x):
    theta = np.radians(x['rotation_angle'][0] - 90)
    x1, y = x['middle_neck','x'], x['middle_neck','y']

    c, s = np.cos(theta), np.sin(theta)
    rot = np.matrix([[c, s], [-s, c]])

    rotated = np.dot(rot, [x1, y])

    # TODO: rotate all body parts
    x['middle_neck', 'x'] = rotated[0, 0]
    x['middle_neck', 'y'] = rotated[0, 1] # this contains the value I want to set

    return x

In [ ]:
# export
def apply_rotation(df):
    df = df.copy()
    df.apply(_rotate_row, axis=1)
    return df

Finally, we can rotate all relative coordinates by this angle around the origin (which is the `body`), so we stabilize (and normalize) the pigeon orientation.

In [ ]:
df_example = df_result.copy()
df_rotation_applied = apply_rotation(df_example)

test_close(df_rotation_applied['middle_neck','x'][0], 0, 0.1)
test_close(df_rotation_applied['middle_neck','y'][0], 1.6, 0.1)